# How To Convert MATLAB File Into PNG File?

### Image Segmentations are mosty developed by skilled people in `.mat` format in MATLAB. We cannot visualize or process those images as such. We may need to convert them suitably into a readable, yet uncompressed, format such as PNG or something else. 

## Import

In [ ]:
# to manage paths
import os
# to process images
import cv2
# to read MATLAB file
from scipy import io
# for array operations
import numpy as np
# for final dataframe outputs
import pandas as pd
# to display images
import matplotlib.pyplot as plt
# to display image legends
import matplotlib as mpl
# to visualize iterations
from tqdm import tqdm
# to process tensors
import tensorflow as tf

# MATLAB files

### We use open source image segmentation files and their corresponding original images (with a Special Thanks!).The open source collection is available at https://github.com/bearpaw/clothing-co-parsing


### Acknowledgement:
> @inproceedings{yang2014clothing,
  title={Clothing Co-Parsing by Joint Image Segmentation and Labeling},
  author={Yang, Wei and Luo, Ping and Lin, Liang}
  booktitle={Computer Vision and Pattern Recognition (CVPR), 2014 IEEE Conference on},
  year={2013},
  organization={IEEE}
}



In [ ]:
!git clone https://github.com/bearpaw/clothing-co-parsing.git

### Have a Look at the downloaded contents

In [ ]:
!ls -p clothing-co-parsing

### `photos/` directory and `annotations/` directory are the images and segmentations collections respectively. 

# Read Images, Segmentations and Labels

### There are 1000 images of people fashion and clothing - and corresponding 1000 segmentation mask images. The image files are in JPG format and masks are in MATLAB format. The files do not have consistency in size (variable height). However, image and mask pairs do have identical sizes. We read the files and write them into tensors for further processing.

### 1. Read Images

In [ ]:
# a list to store image tensors
images = []
# read 1000 images
for i in range(1,1001):
    url = './clothing-co-parsing/photos/%04d.jpg'%(i)
    # use OpenCV for lossless reading
    img = cv2.imread(url, 1)
    # convert BGR image into RGB image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # convert into a tensor
    img = tf.convert_to_tensor(img)
    # resize the image by either cropping or padding with zeros
    img = tf.image.resize_with_crop_or_pad(img,825,550)
    # add to the list
    images.append(img)

In [ ]:
# sample an image and visualize it
plt.figure(figsize=(4,7))
example_image = images[0]
plt.imshow(example_image)
plt.show()

### 2. Segmentations

In [ ]:
# a list to collect mask tensors
segmentations = []
# read 1000 files
for i in range(1,1001):
    url = './clothing-co-parsing/annotations/pixel-level/%04d.mat'%(i)
    # read MATLAB file as image
    file = io.loadmat(url)
    # convert into a tensor
    mask = tf.convert_to_tensor(file['groundtruth'])
    # resize expects 3D image, but we got 2D grayscale image 
    # so expand dimensions
    mask = tf.expand_dims(mask,-1)
    # resize by either cropping excess or padding with zeros
    mask = tf.image.resize_with_crop_or_pad(mask,825,550)
    # append the mask image to the list
    segmentations.append(mask)

In [ ]:
# sample a mask and visualize it
example_seg = segmentations[0]
plt.figure(figsize=(5,7))
plt.imshow(example_seg, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
# what are the unique pixel values?
np.unique(example_seg)

### 3. Read Labels

In [ ]:
label_url = './clothing-co-parsing/label_list'
# read the labels list MATLAB file
label_file = io.loadmat(label_url)['label_list']
# what is its shape?
label_file.shape

In [ ]:
# remove unnecessary dimension
label_file = np.squeeze(label_file)
label_file.shape

In [ ]:
# view the file
label_file

### Each label is an array. Extract the label strings alone. 

In [ ]:
labels = [label[0].astype(str) for label in label_file]
labels

In [ ]:
# sample a mask and show the class labels as legends
plt.figure(figsize=(7,7))
example_seg = segmentations[0]
# obtain unique values (the class numbers)
annotations = np.unique(example_seg.numpy().ravel())
# read the names
names = [labels[a] for a in annotations]
# the values range from 0 to 58, hence normalize for homogeneity
NORM = mpl.colors.Normalize(vmin=0, vmax=58)
# visulaize the image
plt.imshow(example_seg, cmap='jet', norm=NORM)
plt.axis('off')
# prepare patches for legends
PATCHES = [mpl.patches.Patch(color=mpl.cm.jet(NORM(a)), label=f'{a}: {names[i]}') for i,a in enumerate(annotations)]
plt.legend(handles=PATCHES, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### We have read necessary contents from the source file directory. We can delete the directory and its contents to save memory and get clean outputs finally.

In [ ]:
!rm -r clothing-co-parsing

# Display Images, Segmentations and Labels

In [ ]:
plt.figure(figsize=(16,16))
# display 9 images and corresponding masks with legends
for i in range(1,9):
    # display a mask with legends
    plt.subplot(4,4,2*i-1)
    example_seg = segmentations[i]
    annotations = np.unique(example_seg.numpy().ravel())
    names = [labels[a] for a in annotations]
    NORM = mpl.colors.Normalize(vmin=0, vmax=58)
    plt.imshow(example_seg, cmap='jet', norm=NORM)
    plt.axis('off')
    PATCHES = [mpl.patches.Patch(color=mpl.cm.jet(NORM(a)), label=names[i]) for i,a in enumerate(annotations)]
    plt.legend(handles=PATCHES, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
    # display an image
    plt.subplot(4,4,2*i)
    example_image = images[i]
    plt.imshow(example_image)
    plt.axis('off')
    
plt.show()

# Convert Images and Segmentations into PNG File

### PNG images are the uncompressed formats, hence lossless. Obtaining a lossless transition is crucial becuase the segmentation masks are made with integers that refer to the object classes. In the earlier versions of this notebook, conversion was made into JPEG formats. Though JPEG yields compressed and compact outputs, the segmentation classes are compromised. For instance, the conversion creates new classes in the masks, despite there are no such classes in the original masks. Learning Segmentations may become complicated and misleading with such lossy transitions. Hence latest versions of this notebook prefer conversions into PNG formats.

### 1. Convert Images

In [ ]:
for i in tqdm(range(1000)):
    img = images[i]
    # encode into PNG
    img = tf.io.encode_png(img)
    # create a path
    path = os.path.join('IMAGES','img_%04d.png'%(i+1))
    file_name = tf.constant(path)
    # write the PNG file
    tf.io.write_file(file_name, img)

### Check for proper encoding

In [ ]:
# a list to store image paths
PNG_IMAGES = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if 'img_' in file:
            # get image paths
            path = os.path.join(root, file)
            PNG_IMAGES.append(path)

In [ ]:
# filenames are unsorted while writing so sort them now
PNG_IMAGES.sort()
print(PNG_IMAGES[:10])

In [ ]:
# sample an image
im = tf.io.read_file(PNG_IMAGES[102])
# decode it into a tensor
dec = tf.io.decode_png(im, channels=3, dtype=tf.dtypes.uint8)
# visualize it
plt.figure(figsize=(4,7))
plt.imshow(dec)
plt.show()

### 2. Convert Segmentations

In [ ]:
# encode 1000 masks into PNG files
for i in tqdm(range(1000)):
    seg = segmentations[i]
    # encode the tensor into PNG
    seg = tf.io.encode_png(seg)
    # create a path to write
    path = os.path.join('MASKS','seg_%04d.png'%(i+1))
    file_name = tf.constant(path)
    # write the PNG file
    tf.io.write_file(file_name, seg)

In [ ]:
# a list to store the mask paths
PNG_MASKS = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if 'seg_' in file:
            # obtain the path
            path = os.path.join(root, file)
            PNG_MASKS.append(path)

In [ ]:
# paths are unsorted while writing
PNG_MASKS.sort()
# view some paths
print(PNG_MASKS[:10])

### Check for proper encoding

In [ ]:
# read a sample mask
im = tf.io.read_file(PNG_MASKS[102])
# decode into a tensor
dec = tf.io.decode_png(im, channels=0, dtype = tf.dtypes.uint8)
# visualize the mask
plt.figure(figsize=(5,7))
plt.imshow(dec, cmap='jet', norm=NORM)
plt.colorbar()
plt.show()

# Write Files to Output

### Zip Images and Segmentations (helpful for download!)

In [ ]:
!zip -r -q png_images.zip IMAGES/
!zip -r -q png_masks.zip MASKS/

### Keep a Few Images and Segmentations and Remove the rest (for clean output, otherwise, the output pane will be overcrowded!)

In [ ]:
for i in range(6,1000):
    # define the paths
    image_path = './IMAGES/img_%04d.png'%(i)
    mask_path = './MASKS/seg_%04d.png'%(i)
    # delete the image
    if os.path.exists(image_path):
        os.remove(image_path)
    
    # delete the mask
    if os.path.exists(mask_path):
        os.remove(mask_path)

### Write the class labels names into a CSV file

In [ ]:
labels = np.array(labels)
labels = pd.Series(labels, name='label_list')
labels.to_csv('labels.csv')

### The downloaded PNG files (and the JPEG files from earlier versions of this notebook) and the labels list CSV file are available as a Kaggle Dataset at [https://www.kaggle.com/rajkumarl/people-clothing-segmentation](https://www.kaggle.com/rajkumarl/people-clothing-segmentation). 

### A Notebook on this dataset is available at [https://www.kaggle.com/rajkumarl/get-started-with-semantic-segmentation](https://www.kaggle.com/rajkumarl/get-started-with-semantic-segmentation)

### Thank You For Your Time!